In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import numpy as np
from numpy import argmax
import re

from sklearn.preprocessing import LabelEncoder
from google.colab import drive 
drive.mount('/content/gdrive')
path_to_csv = '/content/gdrive/My Drive/Sentiment.csv'

data = pd.read_csv(path_to_csv)
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
291/291 - 54s - loss: 0.8241 - accuracy: 0.6444 - 54s/epoch - 185ms/step
144/144 - 4s - loss: 0.7441 - accuracy: 0.6763 - 4s/epoch - 25ms/step
0.744053304195404
0.6762778759002686
['loss', 'accuracy']


In [2]:
statement = 'A lot of good things are happening. We are respected again throughout the world, and that\'s a great thing.@realDonaldTrump'
statement = tokenizer.texts_to_sequences(statement)
statement = pad_sequences(statement, maxlen=28, dtype='int32', value=0)
sentiment_pred = model.predict(statement, batch_size=1, verbose=2)[0]
sentiment = np.argmax(sentiment_pred)
if sentiment == 0:
    res = "Neutral"
elif sentiment < 0:
    res = "Negative"
elif sentiment > 0:
    res = "Positive"
else:
    res = "Cannot be determined"
print('The sentiment of the statement is: ',res)

122/122 - 2s - 2s/epoch - 19ms/step
The sentiment of the statement is:  Positive


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [5] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-4-6d2a9a346835>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


1487/1487 - 195s - loss: 0.8204 - accuracy: 0.6460 - 195s/epoch - 131ms/step
372/372 - 5s - loss: 0.7594 - accuracy: 0.6864 - 5s/epoch - 15ms/step
